In [71]:
import numpy as np
import pandas as pd
import emoji

from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Embedding
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.metrics import f1_score


In [72]:
data = pd.read_csv("emoji_data.csv", header=None)
data.head()

,0,1
0,French macaroon is so tasty,4
1,work is horrible,3
2,I am upset,3
3,throw the ball,1
4,Good joke,2


In [73]:
emoji_dictionary = {
    0: ":red_heart:",
    1: ":baseball:",
    2: ":grinning_face_with_big_eyes:",
    3: ":disappointed_face:",
    4: ":fork_and_knife_with_plate:"
}

def label_to_emoji(label):
    return emoji.emojize(emoji_dictionary[label])

In [74]:
X = data[0].values
Y = data[1].values

Y

array([4, 3, 3, 1, 2, 1, 4, 3, 4, 1, 3, 3, 2, 2, 4, 3, 2, 3, 3, 1, 3, 2,
       2, 2, 0, 1, 0, 4, 2, 0, 2, 0, 0, 3, 4, 0, 2, 1, 3, 1, 0, 4, 0, 3,
       0, 4, 2, 3, 4, 2, 2, 3, 0, 2, 2, 3, 2, 3, 2, 2, 3, 3, 0, 2, 3, 0,
       2, 0, 0, 2, 3, 2, 4, 1, 3, 3, 0, 0, 3, 2, 0, 3, 0, 2, 2, 4, 2, 2,
       0, 0, 2, 3, 0, 4, 2, 1, 2, 3, 3, 2, 3, 0, 3, 0, 2, 0, 2, 3, 4, 3,
       1, 3, 4, 3, 2, 3, 3, 3, 1, 4, 4, 2, 2, 1, 1, 2, 3, 2, 3, 4, 2, 3,
       0, 2, 0, 0, 4, 3, 4, 2, 3, 2, 3, 4, 2, 1, 2, 4, 3, 1, 3, 2, 3, 2,
       2, 3, 3, 2, 4, 0, 0, 0, 3, 0, 0, 1, 1, 2, 2, 2, 0, 3, 2, 3, 3, 1,
       2, 2, 4, 2, 3, 1, 2], dtype=int64)

Embeddings

In [75]:
with open('data/glove_dataset/glove.6B.100d.txt','r', encoding='utf8') as file:
    content = file.readlines()

In [76]:
embeddings = {}

for line in content:
    line = line.split()
    embeddings[line[0]] = np.array(line[1:], dtype=float)

convert input text into tokens

In [77]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
word_to_index = tokenizer.word_index


In [78]:
Xtokens = tokenizer.texts_to_sequences(X)


In [79]:
def get_maxlen(data):
    maxlen = 0
    for sent in data:
        maxlen = max(maxlen, len(sent))
    
    return maxlen
maxlen = get_maxlen(Xtokens)

In [80]:
Xtrain = pad_sequences(Xtokens, maxlen = maxlen, padding = 'post', truncating = 'post')

In [81]:
Ytrain = to_categorical(Y)

Model

In [82]:
embed_size = 100

In [83]:
embedding_matrix = np.zeros((len(word_to_index)+1, embed_size))

for word, i in word_to_index.items():
    embed_vector = embeddings[word]
    embedding_matrix[i] = embed_vector

In [84]:
model = Sequential([
    Embedding(input_dim = len(word_to_index)+1,
              output_dim = embed_size,
              input_length=maxlen,
              weights = [embedding_matrix],
              trainable = False
              ),
    LSTM(units = 16, return_sequences=True),
    LSTM(units = 4),
    Dense(5, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [85]:
model.fit(Xtrain, Ytrain, epochs=100)

Epoch 1/100


6/6 [==============================] - 50s 32ms/step - loss: 1.6245 - accuracy: 0.1913
Epoch 2/100
6/6 [==============================] - 0s 42ms/step - loss: 1.5912 - accuracy: 0.3115
Epoch 3/100
6/6 [==============================] - 0s 48ms/step - loss: 1.5702 - accuracy: 0.3607
Epoch 4/100
6/6 [==============================] - 0s 53ms/step - loss: 1.5512 - accuracy: 0.3497
Epoch 5/100
6/6 [==============================] - 0s 40ms/step - loss: 1.5297 - accuracy: 0.3770
Epoch 6/100
6/6 [==============================] - 1s 150ms/step - loss: 1.5086 - accuracy: 0.4153
Epoch 7/100
6/6 [==============================] - 0s 36ms/step - loss: 1.4834 - accuracy: 0.4317
Epoch 8/100
6/6 [==============================] - 0s 44ms/step - loss: 1.4573 - accuracy: 0.4262
Epoch 9/100
6/6 [==============================] - 0s 43ms/step - loss: 1.4261 - accuracy: 0.4754
Epoch 10/100
6/6 [==============================] - 0s 36ms/step - loss: 1.3922 - accuracy: 0.5027
Epoch 11/100
6/6 [===========

In [86]:
test = ["I am trying", "I want to cry", "This is just sad"]
train_labels = [3,3,3]

test_seq = tokenizer.texts_to_sequences(test)
Xtest = pad_sequences(test_seq, maxlen=maxlen, padding='post', truncating='post')


y_pred = model.predict(Xtest)
y_pred = np.argmax(y_pred, axis=1)
y_pred

1/1 [==============================] - 10s 10s/step


array([2, 2, 3], dtype=int64)

In [87]:
for i in range(len(test)):
    print(test[i], label_to_emoji(y_pred[i]))

I am trying 😃
I want to cry 😃
This is just sad 😞


F1 Score


In [88]:
f1 = f1_score(train_labels,y_pred,average='weighted')
print(f"Test F1 Score: {f1:.4f}")

Test F1 Score: 0.5000
